In [1]:
import numpy as np 
import pandas as pd 
import torch
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import time
import copy

# Classifier Libraries
from sklearn.ensemble import IsolationForest
import collections
from sklearn.svm import  SVC
from sklearn.tree import  DecisionTreeClassifier
from sklearn.linear_model import  LogisticRegression
from sklearn.neighbors import  KNeighborsClassifier
import xgboost as xgb
import lightgbm as lgb
# train/test split libraries and preprocessing
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler

from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline

# libraries to handle imbalanced datasets
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, balanced_accuracy_score, cohen_kappa_score, average_precision_score
from collections import Counter

/home/pratyush/miniconda3/envs/cvpr/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('CCF/creditcard.csv')
# train_df = pd.read_csv('custom/train.csv')
# X_test = pd.read_csv('custom/test.csv')
# Y_test = pd.read_csv('custom/test_label.csv')

In [3]:
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [4]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# RobustScaler is less prone to outliers.

# std_scaler = StandardScaler()
rob_scaler = RobustScaler()

df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1,1))

df.drop(['Time','Amount'], axis=1, inplace=True)

In [5]:
scaled_amount = df['scaled_amount']
scaled_time = df['scaled_time']

df.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)

# Amount and Time are Scaled!

df.head()

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.783274,-0.994983,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.269825,-0.994983,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,4.983721,-0.994972,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,1.418291,-0.994972,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,0.670579,-0.994960,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


In [6]:
df

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.783274,-0.994983,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.269825,-0.994983,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,4.983721,-0.994972,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,1.418291,-0.994972,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,0.670579,-0.994960,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,-0.296653,1.034951,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,...,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0
284803,0.038986,1.034963,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,...,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,0
284804,0.641096,1.034975,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,...,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,0
284805,-0.167680,1.034975,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,...,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,0


In [7]:
df['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

# Undersampling

In [75]:
df = df.sample(frac=1)

In [76]:
fraud_df = df.loc[df['Class'] == 1]
non_fraud_df = df.loc[df['Class'] == 0][:492]
undersampled_df = pd.concat([fraud_df, non_fraud_df])
df_new = undersampled_df.sample(frac=1,random_state=42)

In [77]:
df_new

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
241069,-0.223713,0.777464,2.002352,0.075215,-1.618153,0.341536,0.395342,-0.576969,0.063434,-0.064411,...,-0.161534,-0.295119,-0.794828,0.354892,0.604837,-0.335625,0.148495,-0.065255,-0.039087,0
123141,3.252428,-0.092412,-6.616293,3.563428,-7.058901,4.284346,-5.096299,-1.768618,-4.937554,2.748460,...,-0.541634,1.215976,0.041178,-1.059098,0.275662,0.057425,-0.265838,-0.514637,0.388590,1
100456,-0.003773,-0.201741,1.128313,-0.453136,1.232589,0.196365,-1.272134,-0.185062,-0.743131,0.192109,...,-0.057050,-0.162475,-0.281501,0.082957,0.606033,0.068380,0.911764,-0.040723,0.008143,0
226877,-0.307413,0.706622,-6.423306,1.658515,-5.866440,2.052064,-0.615817,-3.372266,-5.036556,2.643106,...,-0.713516,0.641211,-0.256678,-2.337233,-0.158278,1.198797,-0.261258,0.780125,-0.731801,1
42784,0.328233,-0.510450,-10.940739,6.261586,-14.182339,7.183602,-9.951363,-3.860820,-13.547302,7.096472,...,-0.088342,2.267448,-0.492029,-0.239303,0.454368,-0.101611,0.446997,0.062293,-0.439770,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114271,-0.077133,-0.132567,-2.869795,1.335667,-1.009530,1.693885,-0.741480,-0.796773,-2.614242,1.066636,...,0.011557,0.528421,0.228027,-0.749298,-0.067179,0.215792,-0.453651,0.349191,0.198862,1
150601,-0.293440,0.106322,-3.291125,4.401194,-8.394212,4.453580,-4.790055,-4.240182,-9.219001,1.974030,...,1.490869,2.102343,0.597370,-0.328086,0.445752,0.585281,-0.399005,2.116004,1.050744,1
102457,-0.027947,-0.193611,1.487021,-1.072903,0.305312,-1.504080,-1.395459,-0.524906,-1.055509,-0.046381,...,-0.335286,-0.187902,-0.228225,-0.015631,-0.024618,0.376967,-0.202395,0.020213,0.007298,0
150687,-0.293440,0.108143,-10.300820,6.483095,-15.076363,6.554191,-8.880252,-4.471672,-14.900689,3.840170,...,0.113244,1.508748,1.041642,-0.682790,0.573544,-1.602389,-0.393521,-0.468893,0.105920,1


# OVERSAMPLING


In [8]:
from imblearn.over_sampling import SMOTE
X = df.drop(columns=['Class'],axis=1)
y = df['Class']
print(X.shape,y.shape)
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,shuffle=True)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
sm = SMOTE(sampling_strategy='minority', random_state=42)
Xsm_train, ysm_train = sm.fit_resample(X_train, y_train)

(284807, 30) (284807,)
(213605, 30) (71202, 30) (213605,) (71202,)


In [9]:
X_test

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
43428,4.781527,-0.507372,-16.526507,8.584972,-18.649853,9.505594,-13.793819,-2.832404,-16.701694,7.517344,...,3.101735,-1.514923,1.190739,-1.127670,-2.358579,0.673461,-1.413700,-0.462762,-2.018575,-1.042804
49906,6.960386,-0.474994,0.339812,-2.743745,-0.134070,-1.385729,-1.451413,1.015887,-0.524379,0.224060,...,0.432546,0.506044,-0.213436,-0.942525,-0.526819,-1.156992,0.311211,-0.746647,0.040996,0.102038
29474,0.125760,-0.578108,1.399590,-0.590701,0.168619,-1.029950,-0.539806,0.040444,-0.712567,0.002299,...,1.192245,0.212877,0.102398,0.168269,-0.166639,-0.810250,0.505083,-0.232340,0.011409,0.004634
276481,-0.286453,0.968421,-0.432071,1.647895,-1.669361,-0.349504,0.785785,-0.630647,0.276990,0.586025,...,0.101264,-0.244633,0.358932,0.873663,-0.178642,-0.017171,-0.207392,-0.157756,-0.237386,0.001934
278846,-0.294977,0.984281,2.014160,-0.137394,-1.015839,0.327269,-0.182179,-0.956571,0.043241,-0.160746,...,0.404711,-0.255293,-0.238644,-0.616400,0.347045,0.061561,-0.360196,0.174730,-0.078043,-0.070571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234953,-0.293579,0.745920,2.063120,-0.058896,-1.063312,0.419454,-0.161770,-1.223186,0.172994,-0.326309,...,0.144102,-0.236574,-0.291006,-0.710108,0.343945,0.041895,-0.304684,0.196004,-0.071614,-0.060330
235147,-0.064137,0.746931,1.890943,0.253602,-0.286539,3.386498,0.497538,1.296918,-0.496714,0.332509,...,-0.853098,-0.202355,-0.224074,-0.681011,0.251275,-1.535055,-0.431158,-0.311445,0.005588,-0.047177
199459,0.947390,0.567276,1.872463,-0.632348,-1.143998,0.042980,-0.090738,-0.004744,-0.271789,0.131482,...,0.668883,-0.082202,-0.450105,-1.572183,0.440711,0.110062,-0.710272,0.018230,-0.095852,-0.045685
172357,-0.027947,0.427343,-1.338327,1.082228,-1.425917,-0.648609,0.702802,-0.486567,0.254775,0.659526,...,0.242739,-0.224321,-0.359127,-0.872318,0.375686,0.063454,-0.297920,0.094997,-0.304587,-0.213661


In [10]:
Xsm_train = Xsm_train.drop(columns=['scaled_amount','scaled_time'])
X_test = X_test.drop(columns=['scaled_amount','scaled_time'])

In [11]:
# Our data is already scaled we should split our training and test sets
from sklearn.model_selection import train_test_split

# This is explicitly used for undersampling.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
X_train

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
223361,-0.167819,0.689153,1.955041,-0.380783,-0.315013,0.330155,-0.509374,-0.086197,-0.627978,0.035994,...,0.167987,-0.125390,0.238197,0.968305,0.053208,-0.278602,-0.044999,-0.216780,0.045168,-0.047145
165061,0.333962,0.381595,-0.400975,-0.626943,1.555339,-2.017772,-0.107769,0.168310,0.017959,-0.401619,...,-1.692780,-0.470372,-0.153485,0.421703,0.113442,-1.004095,-1.176695,0.361924,-0.370469,-0.144792
238186,-0.139873,0.762145,0.072509,0.820566,-0.561351,-0.709897,1.080399,-0.359429,0.787858,0.117276,...,0.198772,0.012227,-0.314638,-0.872959,0.083391,0.148178,-0.431459,0.119690,0.206395,0.070288
150562,1.333613,0.105476,-0.535045,1.014587,1.750679,2.769390,0.500089,1.002270,0.847902,-0.081323,...,-2.147124,-0.253757,0.063525,0.443431,-0.072754,0.448192,-0.655203,-0.181038,-0.093013,-0.064931
138452,0.052540,-0.023931,-4.026938,1.897371,-0.429786,-0.029571,-0.855751,-0.480406,-0.435632,1.313760,...,0.712695,-0.012320,-0.480691,-0.230369,0.250717,0.066399,0.470787,0.245335,0.286904,-0.322672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,-0.279746,-0.106604,1.173488,0.100792,0.490512,0.461596,-0.296377,-0.213165,-0.165254,0.119221,...,-0.182494,-0.157534,-0.186027,-0.574283,0.161405,-0.006140,0.091444,0.109235,-0.020922,0.003967
259178,0.939426,0.872990,-0.775981,0.144023,-1.142399,-1.241113,1.940358,3.912076,-0.466107,1.360620,...,0.094703,-0.295730,0.037078,-0.019575,0.241830,0.682820,-1.635109,-0.770941,0.066006,0.137056
131932,-0.252358,-0.057531,-0.146609,0.992946,1.524591,0.485774,0.349308,-0.815198,1.076640,-0.395316,...,-0.229614,0.007155,0.052649,0.354089,-0.291198,0.402849,0.237383,-0.398467,-0.121139,-0.196195
146867,-0.293440,0.038053,-2.948638,2.354849,-2.521201,-3.798905,1.866302,2.727695,-0.471769,2.217537,...,-1.143752,0.417396,-0.332759,-1.047514,0.143326,0.678869,0.319710,0.426309,0.496912,0.335822


In [13]:
X_train = X_train.drop(columns=['scaled_amount','scaled_time'])
X_test = X_test.drop(columns=['scaled_amount','scaled_time'])

In [14]:
Xsm_train = Xsm_train.values
X_test = X_test.values


In [18]:
import xgboost as xgb
import lightgbm as lgb
classifiers = {
    "LogisiticRegression": LogisticRegression(solver='saga'),
    # "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "XGBoostClassifier":xgb.XGBClassifier(),
    "LightGBMClassifier":lgb.LGBMClassifier()
}

In [19]:

for key, classifier in classifiers.items():
    scores = cross_validate(classifier, X_train, y_train, cv=5, scoring=['accuracy', 'f1', 'roc_auc'])
    classifier.fit(X_train,y_train)
    pred = classifier.predict(X_test)
    f1_score_ = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test,pred)
    print("Classifier: ", classifier.__class__.__name__)
    print("F1 score:", round(f1_score_, 4))
    print("ROC AUC:", round(roc_auc, 4))

/home/pratyush/miniconda3/envs/cvpr/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pratyush/miniconda3/envs/cvpr/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pratyush/miniconda3/envs/cvpr/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pratyush/miniconda3/envs/cvpr/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/pratyush/miniconda3/envs/cvpr/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home

Classifier:  LogisticRegression
F1 score: 0.6503
ROC AUC: 0.7703


In [149]:
clf = IsolationForest(random_state=0,n_estimators=200).fit(X_train)
pred = clf.predict(X_test)
predictions = np.where(pred == -1, 0, 1)
f1_score_ = f1_score(y_test,predictions)
roc_auc = roc_auc_score(y_test,predictions)
f1_score_,roc_auc

(0.6028368794326241, 0.38636363636363635)

In [139]:
clf = LocalOutlierFactor(novelty=True).fit(X_train)
pred = clf.predict(X_test)
predictions = np.where(pred == -1, 0, 1)
f1_score_ = f1_score(y_test,predictions)
roc_auc = roc_auc_score(y_test,predictions)
f1_score_,roc_auc

(0.6505190311418685, 0.43876698014629045)